<a href="https://colab.research.google.com/github/JaquelineMera/etl-superstore/blob/main/web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Hacer peticion HTTP
import requests
# Manipular código y guardar datos tabulares en archivo CSV
import pandas as pd

# url de la página web a «escrapear»
url = 'https://en.wikipedia.org/wiki/List_of_supermarket_chains'

# pasar "User-agent" para simular interacción con la página usando Navegador web
headers = {"User-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

respuesta = requests.get(url, headers=headers)

# El código de respuesta <200> indicará que todo salió bien
print(respuesta)

# También puedes imprimir el código de estado
print(respuesta.status_code)


<Response [200]>
200


In [ ]:
all_tables = pd.read_html(respuesta.content, encoding = 'utf8')

In [ ]:
print(f'Total de tablas encontradas: {len(all_tables)}')

Total de tablas encontradas: 3


In [ ]:
# Buscar la tabla que coincide con algun elemento de la tabla
matched_table = pd.read_html(respuesta.text, match='Company')

# Imprime numero de tablas que coinciden con parametro match
print(f'Company: {len(matched_table)}')

Company: 1


<ipython-input-4-5484cf62082a>:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matched_table = pd.read_html(respuesta.text, match='Company')


In [ ]:
# Guardar tabla en variable con nombre semántico
multinacional = matched_table[0]

# Verificamos si es la tabla que buscamos
multinacional.tail(5)

,Company,Headquarters,Served countries (besides the headquarters),Map,Number of locations,Number of employees,Unnamed: 6
369,T&T Supermarket,Canada,NaN,NaN,33,NaN,NaN
370,Match,Belgium,"Luxembourg, France",NaN,217,NaN,NaN
371,C-market,Serbia,NaN,NaN,NaN,NaN,NaN
372,Tegut,Germany,NaN,NaN,275,7700,NaN
373,Comet,United Kingdom,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Imprimir la tabla
print(multinacional.head())

          Company         Headquarters  \
0        7-Eleven  Japan United States   
1            Aeon                Japan   
2  Ahold Delhaize          Netherlands   
3       Aldi Nord              Germany   
4        Aldi Süd              Germany   

         Served countries (besides the headquarters)  Map Number of locations  \
0  Australia, Canada, China, Cambodia, Denmark, H...  NaN               84500   
1  Australia, Cambodia, China, Hong Kong, India, ...  NaN               20008   
2  Belgium (as Albert Heijn and Delhaize), Czech ...  NaN                7659   
3  Belgium, Denmark, France, Luxembourg, Netherla...  NaN                5241   
4  Australia, Austria (as Hofer), China, Hungary,...  NaN                7178   

  Number of employees  Unnamed: 6  
0              170000         NaN  
1            560,000+         NaN  
2              375000         NaN  
3               72811         NaN  
4              201361         NaN  


In [ ]:
# Eliminar las columnas 'Map' y 'Unnamed: 6'
multinacional_clean = multinacional.drop(columns=['Map', 'Unnamed: 6'])

In [ ]:
# Renombrar las columnas del DataFrame multinacional_clean
multinacional_clean = multinacional_clean.rename(columns={
    'Company': 'company',
    'Headquarters': 'headquarters',
    'Served countries (besides the headquarters)': 'served_countries',
    'Number of locations': 'number_of_locations',
    'Number of employees': 'number_of_employees'
})

In [ ]:
# Imprimir la tabla sin columnas
print(multinacional_clean.head())

          company         headquarters  \
0        7-Eleven  Japan United States   
1            Aeon                Japan   
2  Ahold Delhaize          Netherlands   
3       Aldi Nord              Germany   
4        Aldi Süd              Germany   

                                    served_countries number_of_locations  \
0  Australia, Canada, China, Cambodia, Denmark, H...               84500   
1  Australia, Cambodia, China, Hong Kong, India, ...               20008   
2  Belgium (as Albert Heijn and Delhaize), Czech ...                7659   
3  Belgium, Denmark, France, Luxembourg, Netherla...                5241   
4  Australia, Austria (as Hofer), China, Hungary,...                7178   

  number_of_employees  
0              170000  
1            560,000+  
2              375000  
3               72811  
4              201361  


In [ ]:
# Código para crear Índice de la tabla
# multinacional_clean.set_index('Company', inplace = True)

# Verificamos el cambio de índice
# multinacional_clean.head()

In [ ]:
from unicodedata import normalize

In [ ]:
def remove_whitespace(x):
    """Funcion para normalizar datos con Unicode para luego quitar los espacios usando .replace().

    Argumentos de entrada: Nombre de columna o lista con nombres de columnas.
    Retorna: columna o columnas sin espacios en blanco
    """
    if isinstance(x, str):
        return normalize('NFKC', x).replace(' ', '')
    else:
        return x

In [ ]:
# Mostrar tipo de datos de la tabla
multinacional_clean.head()

,company,headquarters,served_countries,number_of_locations,number_of_employees
0,7-Eleven,Japan United States,"Australia, Canada, China, Cambodia, Denmark, H...",84500,170000
1,Aeon,Japan,"Australia, Cambodia, China, Hong Kong, India, ...",20008,"560,000+"
2,Ahold Delhaize,Netherlands,"Belgium (as Albert Heijn and Delhaize), Czech ...",7659,375000
3,Aldi Nord,Germany,"Belgium, Denmark, France, Luxembourg, Netherla...",5241,72811
4,Aldi Süd,Germany,"Australia, Austria (as Hofer), China, Hungary,...",7178,201361


In [ ]:
multinacional_clean.dtypes

,0
company,object
headquarters,object
served_countries,object
number_of_locations,object
number_of_employees,object


In [ ]:
import re

# Función para limpiar los datos numéricos
def clean_numeric_column(value):
    if isinstance(value, str):
        # Eliminar el símbolo '+'
        value = value.replace("+", "")
        # Eliminar comas
        value = value.replace(",", "")
        # Eliminar el texto dentro de paréntesis
        value = re.sub(r"\(.*?\)", "", value)
        # Mantener solo el primer número, eliminando espacios en blanco
        value = value.strip()

    return value

# Aplicar la limpieza a las columnas numéricas
numeric_cols = ['number_of_locations', 'number_of_employees']
multinacional_clean[numeric_cols] = multinacional_clean[numeric_cols].applymap(clean_numeric_column)

# Mostrar el DataFrame resultante
print(multinacional_clean)

             company         headquarters  \
0           7-Eleven  Japan United States   
1               Aeon                Japan   
2     Ahold Delhaize          Netherlands   
3          Aldi Nord              Germany   
4           Aldi Süd              Germany   
..               ...                  ...   
369  T&T Supermarket               Canada   
370            Match              Belgium   
371         C-market               Serbia   
372            Tegut              Germany   
373            Comet       United Kingdom   

                                      served_countries number_of_locations  \
0    Australia, Canada, China, Cambodia, Denmark, H...               84500   
1    Australia, Cambodia, China, Hong Kong, India, ...               20008   
2    Belgium (as Albert Heijn and Delhaize), Czech ...                7659   
3    Belgium, Denmark, France, Luxembourg, Netherla...                5241   
4    Australia, Austria (as Hofer), China, Hungary,...               

<ipython-input-16-0fd0299ba07b>:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  multinacional_clean[numeric_cols] = multinacional_clean[numeric_cols].applymap(clean_numeric_column)


In [ ]:
# Convertir las columnas a numérico
multinacional_clean[numeric_cols] = multinacional_clean[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Verificar los tipos de datos
print(multinacional_clean.dtypes)

company                 object
headquarters            object
served_countries        object
number_of_locations    float64
number_of_employees    float64
dtype: object


In [ ]:
# Verificamos
multinacional_clean.head()

,company,headquarters,served_countries,number_of_locations,number_of_employees
0,7-Eleven,Japan United States,"Australia, Canada, China, Cambodia, Denmark, H...",84500.0,170000.0
1,Aeon,Japan,"Australia, Cambodia, China, Hong Kong, India, ...",20008.0,560000.0
2,Ahold Delhaize,Netherlands,"Belgium (as Albert Heijn and Delhaize), Czech ...",7659.0,375000.0
3,Aldi Nord,Germany,"Belgium, Denmark, France, Luxembourg, Netherla...",5241.0,72811.0
4,Aldi Süd,Germany,"Australia, Austria (as Hofer), China, Hungary,...",7178.0,201361.0


In [ ]:
# Creamos diccionario y pasamos múltiples columnas con el tipo de dato a asignar
convert_dict = {
    'company': 'string',
    'headquarters': 'string',
    'served_countries': 'string',
    'number_of_locations': 'float64',
    'number_of_employees': 'float64'
}

# Convertimos los tipos de datos de las columnas utilizando el diccionario
multinacional_clean = multinacional_clean.astype(convert_dict)

# Verificamos que las columnas con números tengan el tipo de dato numérico asignado
print(multinacional_clean.dtypes)

company                string[python]
headquarters           string[python]
served_countries       string[python]
number_of_locations           float64
number_of_employees           float64
dtype: object


In [ ]:
# Guarda el DataFrame a archivo CSV
multinacional_clean.to_csv('multinacional_clean.csv', index=False)

# Leer el archivo CSV que se acaba de crear
df = pd.read_csv('multinacional_clean.csv')

# Verificamos las primeras 3 filas del archivo leído
print(df.head(3))


          company         headquarters  \
0        7-Eleven  Japan United States   
1            Aeon                Japan   
2  Ahold Delhaize          Netherlands   

                                    served_countries  number_of_locations  \
0  Australia, Canada, China, Cambodia, Denmark, H...              84500.0   
1  Australia, Cambodia, China, Hong Kong, India, ...              20008.0   
2  Belgium (as Albert Heijn and Delhaize), Czech ...               7659.0   

   number_of_employees  
0             170000.0  
1             560000.0  
2             375000.0  


In [ ]:
# Cargar o descargar archivos
from google.colab import files

# Descarga archivo con datos de tabla
files.download("multinacional_clean.csv")

print('Listo, en un momento saldrá la opción "Guardar Como" para descargar el archivo...')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Listo, en un momento saldrá la opción "Guardar Como" para descargar el archivo...
